In [ ]:
%matplotlib inline

import xarray as xr
from os.path import join, abspath
import os
from scipy import quantile

RUN = abspath(os.getenv('RUN', '../models/12/test'))


data_2d_path = join(RUN, 'OUT_2D', '*.nc')
data_3d_path = join(RUN, 'OUT_3D', '*.nc')
data_stat_path = join(RUN, 'OUT_STAT', '*.nc')

print(data_2d_path)
data_2d = xr.open_mfdataset(data_2d_path)
data_3d = xr.open_mfdataset(data_3d_path)
stat = xr.open_mfdataset(data_stat_path)

data_3d['FQTNN'] *= 86400
data_3d['FQTNN'].attrs['units'] = 'g/kg/d'
data_3d['FSLINN'] *= 86400
data_3d['FSLINN'].attrs['units'] = 'K/d'

In [ ]:
data_2d.PW[::12].plot(col='time', col_wrap=3)

In [ ]:
data_2d.W500[::12].plot(col='time', col_wrap=3)

# Forcing in a Single Location

In [ ]:
for loc in [dict(x=0, y=32), dict(x=0, y=10)]:
    for key in ['FQTNN', 'FSLINN', 'SLI', 'QT']:
        plt.figure()
        data_3d[key].isel(**loc).plot.contourf(x='time', levels=11)

# Zonal-Means

In [ ]:
data_3d['FQTNN']

In [ ]:
def get_plot_kwargs(x):
    kwargs = {}
    a,b = quantile(x, [0.02, .98])
    if a * b < 0:
        cmap = 'RdBu_r'
        m = max(abs(a), abs(b))
        a = -m 
        b = m
    else:
        cmap = 'viridis'
    return dict(levels=np.linspace(a, b, 11), cmap=cmap)

for key in ['FQTNN', 'FSLINN', 'SLI', 'QT']:
    plt.figure()
    x = data_3d[key][::12].mean('x')
    x.plot.contourf(col='time', **get_plot_kwargs(x))